In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import tensorflow as tf
import random
import json

In [2]:
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
with open("intents.json") as file:
    data = json.load(file)

In [6]:
training_sentences = [] #x
training_labels = [] #y
tags = [] #classes
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in tags:
        tags.append(intent['tag'])

In [7]:
print(training_labels)

['confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_problem', 'confirm_pr

In [8]:
print(tags)

['confirm_problem', 'no_problem', 'call', 'yes_call', 'no_call', 'pain_cream', 'bandaid', 'vaseline', 'fall', 'critical', 'stomach_pain', 'ointment', 'thanks', 'wait', 'done']


In [9]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(training_labels)
training_labels = enc.transform(training_labels)

In [10]:
print(training_labels)

[ 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  7  7  7  7  7  7  7  7  7  1  1
  1  1  1  1  1  1 14 14 14 14 14 14 14  6  6  6  6  6  6  6  6  6  6  6
  6  6  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0 12 12 12 12 12 12 12 12 12 12 12 12  5  5  5  5  5  5  5  5  5
  5  5  5  5  5  5  5  5  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3 10 10 10 10 10 10 10 10 10  8  8  8  8  8  8  8  8
  8  8  8  8 11 11 11 11 11 11 13 13 13 13 13 13 13 13 13 13 13  4  4]


In [11]:
vocab_size = 10000
max_len = 200
trunc_type = 'post'
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, truncating=trunc_type, maxlen=max_len)

In [12]:
classes = len(tags)
print(classes)

15


In [43]:
model = tf.keras.models.Sequential()
embed_size = 128
model.add(keras.layers.Embedding(vocab_size, embed_size, input_length=max_len))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(classes, activation='softmax'))

In [33]:
inp = keras.layers.Input(shape=(max_len, ))
embed_size = 128
x = keras.layers.Embedding(vocab_size, embed_size)(inp)
x = keras.layers.LSTM(60, return_sequences=True,name='lstm_layer')(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(50, activation="relu")(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(15, activation="sigmoid")(x)
model = keras.models.Model(inputs=inp, outputs=x)

In [42]:
from keras_visualizer import visualizer

model = keras.models.Sequential([  
    keras.layers.Dense(64, activation='relu', input_shape=(8,)),  
    keras.layers.Dense(6, activation='softmax'),  
    keras.layers.Dense(32),  
    keras.layers.Dense(9, activation='sigmoid')])  
visualizer(model, format='png', view=True)

ValueError: Keras Visualizer: Error while visualizing

In [28]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 200, 128)          1280000   
_________________________________________________________________
lstm_layer (LSTM)            (None, 200, 60)           45360     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 60)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 50)                3050      
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0   

In [44]:
training_labels_final = np.array(training_labels)

In [45]:
training_labels_final.shape

(263,)

In [47]:
EPOCHS = 500
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(padded, training_labels_final, epochs=EPOCHS)

Epoch 1/500
9/9 [==============================] - 1s 24ms/step - loss: 2.8436 - accuracy: 0.6518
Epoch 2/500
9/9 [==============================] - 0s 24ms/step - loss: 0.8701 - accuracy: 0.7307
Epoch 3/500
9/9 [==============================] - 0s 23ms/step - loss: 0.6726 - accuracy: 0.7237
Epoch 4/500
9/9 [==============================] - 0s 23ms/step - loss: 0.4493 - accuracy: 0.8452
Epoch 5/500
9/9 [==============================] - 0s 24ms/step - loss: 0.2146 - accuracy: 0.9159
Epoch 6/500
9/9 [==============================] - 0s 23ms/step - loss: 0.2045 - accuracy: 0.9067
Epoch 7/500
9/9 [==============================] - 0s 22ms/step - loss: 0.1801 - accuracy: 0.9177
Epoch 8/500
9/9 [==============================] - 0s 23ms/step - loss: 0.1802 - accuracy: 0.9323
Epoch 9/500
9/9 [==============================] - 0s 23ms/step - loss: 0.1747 - accuracy: 0.9296
Epoch 10/500
9/9 [==============================] - 0s 24ms/step - loss: 0.1971 - accuracy: 0.9105
Epoch 11/500
9/9 [=

9/9 [==============================] - 0s 23ms/step - loss: 1.3299 - accuracy: 0.6175
Epoch 84/500
9/9 [==============================] - 0s 23ms/step - loss: 0.4229 - accuracy: 0.8279
Epoch 85/500
9/9 [==============================] - 0s 23ms/step - loss: 0.2822 - accuracy: 0.8593
Epoch 86/500
9/9 [==============================] - 0s 23ms/step - loss: 0.2992 - accuracy: 0.8500
Epoch 87/500
9/9 [==============================] - 0s 23ms/step - loss: 0.2793 - accuracy: 0.8796
Epoch 88/500
9/9 [==============================] - 0s 22ms/step - loss: 0.2274 - accuracy: 0.8814
Epoch 89/500
9/9 [==============================] - 0s 23ms/step - loss: 0.2360 - accuracy: 0.8996
Epoch 90/500
9/9 [==============================] - 0s 23ms/step - loss: 0.3209 - accuracy: 0.8568
Epoch 91/500
9/9 [==============================] - 0s 23ms/step - loss: 0.2231 - accuracy: 0.9073
Epoch 92/500
9/9 [==============================] - 0s 23ms/step - loss: 0.3460 - accuracy: 0.8439
Epoch 93/500
9/9 [=====

9/9 [==============================] - 0s 23ms/step - loss: 0.1881 - accuracy: 0.9145
Epoch 166/500
9/9 [==============================] - 0s 23ms/step - loss: 0.1612 - accuracy: 0.9453
Epoch 167/500
9/9 [==============================] - 0s 22ms/step - loss: 0.1455 - accuracy: 0.9353
Epoch 168/500
9/9 [==============================] - 0s 23ms/step - loss: 0.1507 - accuracy: 0.9249
Epoch 169/500
9/9 [==============================] - 0s 23ms/step - loss: 0.1681 - accuracy: 0.9256
Epoch 170/500
9/9 [==============================] - 0s 23ms/step - loss: 0.1587 - accuracy: 0.9311
Epoch 171/500
9/9 [==============================] - 0s 23ms/step - loss: 0.1561 - accuracy: 0.9214
Epoch 172/500
9/9 [==============================] - 0s 23ms/step - loss: 0.1702 - accuracy: 0.9264
Epoch 173/500
9/9 [==============================] - 0s 23ms/step - loss: 0.1533 - accuracy: 0.9424
Epoch 174/500
9/9 [==============================] - 0s 23ms/step - loss: 0.1684 - accuracy: 0.9090
Epoch 175/500


9/9 [==============================] - 0s 24ms/step - loss: 0.1608 - accuracy: 0.9153
Epoch 248/500
9/9 [==============================] - 0s 24ms/step - loss: 0.2234 - accuracy: 0.8813
Epoch 249/500
9/9 [==============================] - 0s 25ms/step - loss: 0.3557 - accuracy: 0.8273
Epoch 250/500
9/9 [==============================] - 0s 24ms/step - loss: 0.5603 - accuracy: 0.7717
Epoch 251/500
9/9 [==============================] - 0s 25ms/step - loss: 0.4650 - accuracy: 0.8175
Epoch 252/500
9/9 [==============================] - 0s 31ms/step - loss: 0.9903 - accuracy: 0.7463
Epoch 253/500
9/9 [==============================] - 0s 25ms/step - loss: 0.6842 - accuracy: 0.7644
Epoch 254/500
9/9 [==============================] - 0s 31ms/step - loss: 0.9931 - accuracy: 0.6405
Epoch 255/500
9/9 [==============================] - 0s 33ms/step - loss: 0.2983 - accuracy: 0.8944
Epoch 256/500
9/9 [==============================] - 0s 27ms/step - loss: 0.4291 - accuracy: 0.8102
Epoch 257/500


9/9 [==============================] - 0s 22ms/step - loss: 0.1456 - accuracy: 0.9369
Epoch 330/500
9/9 [==============================] - 0s 22ms/step - loss: 0.1607 - accuracy: 0.9142
Epoch 331/500
9/9 [==============================] - 0s 22ms/step - loss: 0.1381 - accuracy: 0.9300
Epoch 332/500
9/9 [==============================] - 0s 22ms/step - loss: 0.1267 - accuracy: 0.9341
Epoch 333/500
9/9 [==============================] - 0s 22ms/step - loss: 0.1452 - accuracy: 0.9320
Epoch 334/500
9/9 [==============================] - 0s 22ms/step - loss: 0.1471 - accuracy: 0.9350
Epoch 335/500
9/9 [==============================] - 0s 22ms/step - loss: 0.1530 - accuracy: 0.9449
Epoch 336/500
9/9 [==============================] - 0s 23ms/step - loss: 0.1843 - accuracy: 0.9039
Epoch 337/500
9/9 [==============================] - 0s 23ms/step - loss: 0.1394 - accuracy: 0.9491
Epoch 338/500
9/9 [==============================] - 0s 22ms/step - loss: 0.1314 - accuracy: 0.9397
Epoch 339/500


9/9 [==============================] - 0s 22ms/step - loss: 0.1831 - accuracy: 0.9240
Epoch 412/500
9/9 [==============================] - 0s 22ms/step - loss: 0.1600 - accuracy: 0.9321
Epoch 413/500
9/9 [==============================] - 0s 22ms/step - loss: 0.1378 - accuracy: 0.9294
Epoch 414/500
9/9 [==============================] - 0s 22ms/step - loss: 0.1425 - accuracy: 0.9241
Epoch 415/500
9/9 [==============================] - 0s 24ms/step - loss: 0.1616 - accuracy: 0.9212
Epoch 416/500
9/9 [==============================] - 0s 22ms/step - loss: 0.2444 - accuracy: 0.8846
Epoch 417/500
9/9 [==============================] - 0s 22ms/step - loss: 0.2362 - accuracy: 0.9038
Epoch 418/500
9/9 [==============================] - 0s 22ms/step - loss: 0.2638 - accuracy: 0.8746
Epoch 419/500
9/9 [==============================] - 0s 23ms/step - loss: 0.7837 - accuracy: 0.7469
Epoch 420/500
9/9 [==============================] - 0s 23ms/step - loss: 5.3365 - accuracy: 0.4893
Epoch 421/500


9/9 [==============================] - 0s 23ms/step - loss: 0.1801 - accuracy: 0.9117
Epoch 494/500
9/9 [==============================] - 0s 23ms/step - loss: 0.1665 - accuracy: 0.9135
Epoch 495/500
9/9 [==============================] - 0s 25ms/step - loss: 0.1873 - accuracy: 0.9177
Epoch 496/500
9/9 [==============================] - 0s 23ms/step - loss: 0.1617 - accuracy: 0.9279
Epoch 497/500
9/9 [==============================] - 0s 23ms/step - loss: 0.1374 - accuracy: 0.9488
Epoch 498/500
9/9 [==============================] - 0s 23ms/step - loss: 0.1504 - accuracy: 0.9246
Epoch 499/500
9/9 [==============================] - 0s 22ms/step - loss: 0.1504 - accuracy: 0.9270
Epoch 500/500
9/9 [==============================] - 0s 23ms/step - loss: 0.1740 - accuracy: 0.9220


In [32]:
from keras_visualizer import visualizer
visualizer(model, format='png', view=True)

ValueError: invalid literal for int() with base 10: '200)'

In [17]:
model.save("chat_bot_v4")

INFO:tensorflow:Assets written to: chat_bot_v4\assets


In [3]:
model = keras.models.load_model("chat_bot_v4")

In [31]:
spoken = input()
preprocessed_string = pad_sequences(tokenizer.texts_to_sequences([spoken]), truncating=trunc_type, maxlen=max_len)
print(preprocessed_string)
result = model.predict(preprocessed_string)[0]
print("result: ", result)
results = [(i,r) for i,r in enumerate(result) if r>0.1]
print(results)

return_list = []
for r in results:
    return_list.append((enc.inverse_transform([r[0]])[0], r[1]))
print("return_list: ", return_list)

i need a doctor
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 5 7 1]]
result:  [6.3673568e-01 1.8748641e-04 7.8448671e-01 3.4626635e-05 4.1229099e-02
 8.3817542e-03 9.0292454e-02 9.1078311e-02 3.7128121e-02 8.0778927e-02
 8.5479021e-04 4.8500299e-03 8.8313222e-04 1.9109967e-01 3.0746254e-01]
[(0, 0.6367357), (2, 0.7844867), (13, 0.19109967), (14, 0.30746254)]
return_list:  [('bandaid', 0.6367357), ('confirm_problem', 0.7844867), ('wait', 0.19109967), ('yes_call', 0.30746254)]


In [19]:
import pyttsx3
import speech_recognition as sr
import winsound

In [20]:
def VoiceChat(show_details = False, ERROR_THRESHOLD = 0.0):

    engine = pyttsx3.init(driverName='sapi5')
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[1].id)
    volume = engine.getProperty('volume')
    engine.setProperty('volume', 10.0)
    rate = engine.getProperty('rate')
    #engine.setProperty('rate', rate - 25)
    
    context = ""

    response = 'Distress detected. Is there a problem?'

    rec = sr.Recognizer()
    while True: 
        print("Bot: ", response)
        engine.say(response)
        engine.runAndWait()
        try:
            response_flag = False
            with sr.Microphone() as source:
                if show_details: print("Speak...")
                
                audio = rec.listen(source)
                
                spoken = rec.recognize_google(audio, language='en-US')

                print("You: ", spoken)
                
                preprocessed_string = pad_sequences(tokenizer.texts_to_sequences([spoken]), truncating=trunc_type, maxlen=max_len)
         
                result = model.predict(preprocessed_string)[0]
                if show_details: print("result: ", result)
                # filter out predictions below a threshold
                results = [[i,r] for i,r in enumerate(result) if r>ERROR_THRESHOLD]
                # sort by strength of probability
                results.sort(key=lambda x: x[1], reverse=True)

                return_list = []
                for r in results:
                    return_list.append((enc.inverse_transform([r[0]])[0], r[1]))
                if show_details: print("return_list: ", return_list)

                if return_list:
                    for category,r in return_list:
                        if response_flag == False:
                            for i in data['intents']:
                                if i['tag']==category:
                                    if context in i['context_filter']:
                                        response = np.random.choice(i['responses'])
                                        context = i["context_set"]
                                        if show_details: print("current context: ", context)
                                        response_flag = True

                if response_flag == False:
                    response = np.random.choice(["Sorry, I didn't understand that.", "Sorry, can you repeat that?", 
                                            "I did not understand what you said. Please repeat it."])
                    print("Bot: ", response)
                    engine.say(response)
                    engine.runAndWait()

                if response == 'Glad I could be of service! Goodbye.':
                    return

        except sr.UnknownValueError:
            continue
        except sr.RequestError as e:
            message = 'My speech recognition service has failed. {0}'
            engine.say(message.format(e))
        except (KeyboardInterrupt, EOFError, SystemExit):
            # Press ctrl-c or ctrl-d on the keyboard to exit
            break

In [21]:
VoiceChat(show_details = True, ERROR_THRESHOLD = 0.01)

Bot:  Distress detected. Is there a problem?
Speak...
You:  can you give me some pain
result:  [5.7482451e-01 9.9859869e-01 5.9203825e-10 2.1980377e-08 2.6927346e-01
 1.4000194e-07 5.1469010e-01 5.2697462e-01 9.9883389e-01 2.9120478e-01
 1.1414744e-05 1.3577222e-05 9.7643095e-01 2.4272501e-03 4.2644178e-08]
return_list:  [('ointment', 0.9988339), ('call', 0.9985987), ('vaseline', 0.97643095), ('bandaid', 0.5748245), ('no_problem', 0.5269746), ('no_call', 0.5146901), ('pain_cream', 0.29120478), ('done', 0.26927346)]
current context:  medicine_given
Bot:  Please check the tray for some lotion. It is useful for treating insect bites. Once done return it to the tray.
Speak...
You:  I want pain cream
result:  [2.4091303e-03 9.7829396e-01 3.3264512e-06 3.2805991e-12 4.0453672e-04
 2.0488814e-07 6.3121575e-01 3.2854837e-01 2.2165507e-01 9.9998033e-01
 9.0891558e-07 2.8593293e-01 8.6544251e-01 2.1769583e-02 6.0843951e-05]
return_list:  [('pain_cream', 0.99998033), ('call', 0.97829396), ('vasel

In [24]:
converter = tf.lite.TFLiteConverter.from_saved_model("chat_bot_v4")
tflite_model = converter.convert()

with open('chatbotv4.tflite', 'wb') as f:
      f.write(tflite_model)

In [ ]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test the model on random input data.
input_data = np.array(pad_sequences(tokenizer.texts_to_sequences(["leg"]), truncating=trunc_type, maxlen=max_len),
                      dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
result = interpreter.get_tensor(output_details[0]['index'])[0]
print(result)

show_details = True
response_flag = False
context = ""
if show_details:
    print("result: ", result)
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(result) if r>0.01]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)

    return_list = []
    for r in results:
        return_list.append((enc.inverse_transform([r[0]])[0], r[1]))
    if show_details: print("return_list: ", return_list)

    if return_list:
        for category,r in return_list:
            if response_flag == False:
                for i in data['intents']:
                    if i['tag']==category:
                        if context in i['context_filter']:
                            response = np.random.choice(i['responses'])
                            print("Bot: ", response)
                            context = i["context_set"]
                            if show_details: print("current context: ", context)
                            response_flag = True